In [106]:
# import the necessary packages
from matplotlib import patches, patheffects
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import json
import os

In [103]:
PASCAL_TRAIN_PATH = "/home/varun/datasets/voc07/train"
PASCAL_TEST_PATH = "home/varun/datasets/voc07/test"
JPEGS_PATH = os.path.sep.join([PASCAL_TRAIN_PATH, "JPEGImages"])
JSON_ANNOS_PATH = os.path.sep.join([PASCAL_TRAIN_PATH, "train07.json"])

IMGS, ANNOS, CATEGS = "images", "annotations", "categories"
FNAME, ID, IMG_ID, CATEG_ID, BBOX = "file_name", "id", "image_id", "category_id", "bbox"

In [51]:
train_json = json.load(open(JSON_ANNOS_PATH, "r"))

In [52]:
# categs: dict{categ_id -> categ_name}
categs = {categ[ID] : categ['name'] for categ in train_json[CATEGS]}

# fnames: dict{img_id -> img_filename}
fnames = {img[ID] : img[FNAME] for img in train_json[IMGS]}

# img_ids: list[img_ids]
img_ids = [img[ID] for img in train_json[IMGS]]

In [105]:
# initialize a dict to store the annotations: {img_id -> [annotations]}
train_annos = {}

# loop through the annotations in the json file
for anno in train_json[ANNOS]:
    # check if the annotation is to be ignored     
    if not anno["ignore"]:
        # add the annotation to the appropriate img_id in the dictionary
        L = train_annos.get(anno[IMG_ID], [])
        L.append((anno[CATEG_ID], anno[BBOX]))
        train_annos[anno[IMG_ID]] = L

# print the total number of images with annotations
print(f"[INFO] total number of images in the training set: {len(train_annos)}")

[INFO] total number of images in the training set: 2501


In [100]:
def draw_outline(o, lw):
    o.set_path_effects([patheffects.Stroke(linewidth = lw, foreground = 'black'), patheffects.Normal()])

def visualize(ax, img, annos, figsize = None):    
    # display the image
    ax.imshow(img)   
    
    # loop through the annotations
    for anno in annos:
        # destructure the annotation         
        categ, bbox = anno
        
        # draw the bbox and outline it
        patch = ax.add_patch(patches.Rectangle(bbox[:2], *bbox[-2:], fill = False, edgecolor = 'white', lw = 2))
        draw_outline(patch, 4)
        
        # write the class label and outline it
        text = ax.text(*bbox[:2], categs[categ], verticalalignment = 'top', color = 'white', fontsize = 14, weight = 'bold')
        draw_outline(text, 1)

def montage(ids):
    # assert that the length of ids is 16
    assert len(ids) == 16, "Length of ids must be 16."
    
    # initialize a matplotlib figure
    fig, ax = plt.subplots(4, 4, figsize = (16, 16))

    # loop through the imgs and visualize them
    for i in range(4):
        for j in range(4):
            # read the image from disk
            img = Image.open(os.path.sep.join([JPEGS_PATH, fnames[ids[i * 4 + j]]]))

            # visualize the image
            visualize(ax[i, j], img, train_annos[ids[i * 4 + j]])

In [114]:
# initialize an empty dataframe
df = pd.DataFrame(train_annos, columns = ["img_id", "filename", "annotations"])

# loop through the annotations
for i, (img_id, anno) in enumerate(train_annos.items()):
    df.loc[i] = [img_id, fnames[img_id], anno]

df.to_csv("../pascal/data/voc07_obj_detect.csv", index = False)